In [1]:
import math
import numpy as np
from IPython.display import clear_output
from tqdm import tqdm_notebook as tqdm

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.color_palette("bright")
import matplotlib as mpl
import matplotlib.cm as cm

import torch
from torch import Tensor
from torch import nn
from torch.nn  import functional as F 
from torch.autograd import Variable
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import time
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
def save_checkpoint(epoch, model, optimizer):
    """
    Save model checkpoint.
    :param epoch: epoch number
    :param model: model
    :param optimizer: optimizer
    """
    state = {'epoch': epoch,
             'model': model,
             'optimizer': optimizer}
    filename = 'checkpoint_odenet18_cutout.pth.tar'
    torch.save(state, filename)

class AverageMeter(object):
    """
    Keeps track of most recent, average, sum, and count of a metric.
    """

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [3]:
# 数据集读取

epochs = 200
pre_epoch = 0
BATCH_SIZE = 128
LR = 0.01

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_dataset = torchvision.datasets.CIFAR10(root='data', train=True, download=False, transform=transform_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

test_dataset = torchvision.datasets.CIFAR10(root='data', train=False, download=False, transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [4]:
def ode_solve(z0, t0, t1, f):
    """
    Simplest Euler ODE initial value solver
    """
    h_max = 0.05
    n_steps = math.ceil((abs(t1 - t0)/h_max).max().item())  #向上取整

    h = (t1 - t0)/n_steps
    t = t0
    z = z0

    for i_step in range(n_steps):
        z = z + h * f(z, t)
        t = t + h
    return z

In [5]:
class ODEF(nn.Module):
    def forward_with_grad(self, z, t, grad_outputs):
        """Compute f and a df/dz, a df/dp, a df/dt"""
        batch_size = z.shape[0]  #矩阵的列数

        out = self.forward(z, t)

        a = grad_outputs
        adfdz, adfdt, *adfdp = torch.autograd.grad(
            (out,), (z, t) + tuple(self.parameters()), grad_outputs=(a),
            allow_unused=True, retain_graph=True
        )    #自动求导
        # grad method automatically sums gradients for batch items, we have to expand them back 
        if adfdp is not None:
            adfdp = torch.cat([p_grad.flatten() for p_grad in adfdp]).unsqueeze(0)  #关于p的参数转化为一维数组并加一个参数
            adfdp = adfdp.expand(batch_size, -1) / batch_size   #加batch_size个行 再除以batch_size
        if adfdt is not None:
            adfdt = adfdt.expand(batch_size, 1) / batch_size
        return out, adfdz, adfdt, adfdp

    def flatten_parameters(self):
        p_shapes = []
        flat_parameters = []
        for p in self.parameters():
            p_shapes.append(p.size())
            flat_parameters.append(p.flatten())
        return torch.cat(flat_parameters)   #所有向量放到一个维度

In [6]:
class ODEAdjoint(torch.autograd.Function):
    @staticmethod
    def forward(ctx, z0, t, flat_parameters, func):
        assert isinstance(func, ODEF)      #判断func是否在ODEF类
        bs, *z_shape = z0.size()   #bs是有几行(数据维度),z_shape有几列(数据数量)
        time_len = t.size(0)  #有几行

        with torch.no_grad():  #即使输入求导，输出不求导
            z = torch.zeros(time_len, bs, *z_shape).to(z0)   #三个维度, 时间*数据
            z[0] = z0
            for i_t in range(time_len - 1):
                z0 = ode_solve(z0, t[i_t], t[i_t+1], func)
                z[i_t+1] = z0
        ctx.func = func
        ctx.save_for_backward(t, z.clone(), flat_parameters)  #为了反向传播保留input全部信息
        return z

    @staticmethod
    def backward(ctx, dLdz):
        """
        dLdz shape: time_len, batch_size, *z_shape
        """
        func = ctx.func
        t, z, flat_parameters = ctx.saved_tensors   #被储存的参数
        time_len, bs, *z_shape = z.size()    #z的时间，维度，数据量
        n_dim = np.prod(z_shape)      #np内元素的乘积 
        n_params = flat_parameters.size(0)  #参数数量，size[0]是有几行
        # Dynamics of augmented system to be calculated backwards in time
        def augmented_dynamics(aug_z_i, t_i):
            """
            tensors here are temporal slices
            t_i - is tensor with size: bs, 1
            aug_z_i - is tensor with size: bs, 2*n_dim + n_params + 1
            """
            #aug_z_i为原方程中的增广状态[z(t1),dL/dz(t1),0(\theta),-dL/dt]
            #t_i的维度？ time_len?
            z_i, a = aug_z_i[:, :n_dim], aug_z_i[:, n_dim:2*n_dim]  # ignore parameters and time  
            #z_i是z(t1)，a是dL/dz
            
            # Unflatten z and a
            z_i = z_i.view(bs, *z_shape)   #括号里面是想要的维度,torch.view()的作用
            a = a.view(bs, *z_shape)
            with torch.set_grad_enabled(True):
                #detach_()将计算图中节点转为叶子节点，也就是将节点.grad_fn设置为none，这样detach_()的前一个节点就不会再与当前变量连接
                t_i = t_i.detach().requires_grad_(True) 
                z_i = z_i.detach().requires_grad_(True)
                func_eval, adfdz, adfdt, adfdp = func.forward_with_grad(z_i, t_i, grad_outputs=a)  # bs, *z_shape
                #func_eval是输出, 其余为a乘以导数
                adfdz = adfdz.to(z_i) if adfdz is not None else torch.zeros(bs, *z_shape).to(z_i)  #torch.to()
                adfdp = adfdp.to(z_i) if adfdp is not None else torch.zeros(bs, n_params).to(z_i)
                adfdt = adfdt.to(z_i) if adfdt is not None else torch.zeros(bs, 1).to(z_i)

            # Flatten f and adfdz
            func_eval = func_eval.view(bs, n_dim)
            adfdz = adfdz.view(bs, n_dim) 
            return torch.cat((func_eval, -adfdz, -adfdp, -adfdt), dim=1)  #输出新的增广状态

        dLdz = dLdz.view(time_len, bs, n_dim)  # flatten dLdz for convenience

        with torch.no_grad():
            ## Create placeholders for output gradients
            # Prev computed backwards adjoints to be adjusted by direct gradients
            adj_z = torch.zeros(bs, n_dim).to(dLdz)
            adj_p = torch.zeros(bs, n_params).to(dLdz)
            # In contrast to z and p we need to return gradients for all times
            adj_t = torch.zeros(time_len, bs, 1).to(dLdz)

            for i_t in range(time_len-1, 0, -1):     #反向传播
                z_i = z[i_t]
                t_i = t[i_t]
                f_i = func(z_i, t_i).view(bs, n_dim)

                # Compute direct gradients
                dLdz_i = dLdz[i_t]
                dLdt_i = torch.bmm(torch.transpose(dLdz_i.unsqueeze(-1), 1, 2), f_i.unsqueeze(-1))[:, 0] 
                #bmm三维tensor乘积 第一个维度一样 后两个维度矩阵乘积
                #torch.transpose(a,1,2)是交换a第2和第3维度
                #torch.unsqueeze(-1)是在最后一个维度上加一个
                #dL/dt = dL/dz*dz/dt 在对应的时间节点求

                # Adjusting adjoints with direct gradients
                adj_z += dLdz_i    #z的伴随就是dL/dz  根据节点调整 图上的
                adj_t[i_t] = adj_t[i_t] - dLdt_i   

                # Pack augmented variable
                aug_z = torch.cat((z_i.view(bs, n_dim), adj_z, torch.zeros(bs, n_params).to(z), adj_t[i_t]), dim=-1) #按照bs拼接

                # Solve augmented system backwards
                aug_ans = ode_solve(aug_z, t_i, t[i_t-1], augmented_dynamics)
                
                # Unpack solved backwards augmented system
                adj_z[:] = aug_ans[:, n_dim:2*n_dim]
                adj_p[:] += aug_ans[:, 2*n_dim:2*n_dim + n_params]   #不需要调整，一直加
                adj_t[i_t-1] = aug_ans[:, 2*n_dim + n_params:]
                #得出来的结果分配到新的增广状态

                del aug_z, aug_ans

            ## Adjust 0 time adjoint with direct gradients
            # Compute direct gradients 

            dLdz_0 = dLdz[0]
            dLdt_0 = torch.bmm(torch.transpose(dLdz_0.unsqueeze(-1), 1, 2), f_i.unsqueeze(-1))[:, 0]

            # Adjust adjoints
            adj_z += dLdz_0
            adj_t[0] = adj_t[0] - dLdt_0
        return adj_z.view(bs, *z_shape), adj_t, adj_p, None    #得到了到t0的增广状态

In [7]:
class NeuralODE(nn.Module):
    def __init__(self, func):
        super(NeuralODE, self).__init__()  #调用父类
        assert isinstance(func, ODEF)
        self.func = func

    def forward(self, z0, t=Tensor([0., 1.]), return_whole_sequence=False):
        t = t.to(z0)
        z = ODEAdjoint.apply(z0, t, self.func.flatten_parameters(), self.func)   #只有前向传播
        if return_whole_sequence:
            return z
        else:
            return z[-1]

In [8]:
def norm(dim):
    return nn.BatchNorm2d(dim)  #norm为batchnormalization函数

def conv3x3(in_feats, out_feats, stride=1):
    return nn.Conv2d(in_feats, out_feats, kernel_size=3, stride=stride, padding=1, bias=False) #二维数据的卷积操作

def add_time(in_tensor, t):
    bs, c, w, h = in_tensor.shape
    return torch.cat((in_tensor, t.expand(bs, 1, w, h)), dim=1)  #增加时间变量

In [9]:
class ConvODEF(ODEF):
    def __init__(self, dim):
        super(ConvODEF, self).__init__()
        self.conv1 = conv3x3(dim + 1, dim)
        self.norm1 = norm(dim)
        self.conv2 = conv3x3(dim + 1, dim)
        self.norm2 = norm(dim)

    def forward(self, x, t):
        xt = add_time(x, t)
        h = self.norm1(torch.relu(self.conv1(xt)))
        ht = add_time(h, t)
        dxdt = self.norm2(torch.relu(self.conv2(ht)))
        return dxdt

In [10]:
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [11]:
class odenet(nn.Module):
    def __init__(self, ResidualBlock, ConvODEF, num_classes=10):
        super(odenet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        func = ConvODEF(64)
        self.layer1 = NeuralODE(func)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)   #strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)    #核为4，步长为4
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


def odeNet18():

    return odenet(ResidualBlock, ConvODEF)

In [12]:
class Cutout(object):
    """Randomly mask out one or more patches from an image.
    Args:
        n_holes (int): Number of patches to cut out of each image.
        length (int): The length (in pixels) of each square patch.
    """
    def __init__(self, n_holes, length):
        self.n_holes = n_holes
        self.length = length

    def __call__(self, img):
        """
        Args:
            img (Tensor): Tensor image of size (C, H, W).
        Returns:
            Tensor: Image with n_holes of dimension length x length cut out of it.
        """
        h = img.size(1)
        w = img.size(2)

        mask = np.ones((h, w), np.float32)

        for n in range(self.n_holes):
            y = np.random.randint(h)
            x = np.random.randint(w)

            y1 = np.clip(y - self.length // 2, 0, h)
            y2 = np.clip(y + self.length // 2, 0, h)
            x1 = np.clip(x - self.length // 2, 0, w)
            x2 = np.clip(x + self.length // 2, 0, w)

            mask[y1: y2, x1: x2] = 0.

        mask = torch.from_numpy(mask)
        mask = mask.expand_as(img)
        img = img * mask

        return img

In [13]:
checkpoint = 'checkpoint_odenet18_cutout.pth.tar'
print_freq = 20

def main():
    """
    Training.
    """
    
    global start_epoch, classes, epoch, checkpoint
    
    # 初始化模型
    
    if checkpoint is None:
        start_epoch = 0
        model = odeNet18()
        optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
    else:
        checkpoint = torch.load(checkpoint, map_location = 'cuda')
        start_epoch = checkpoint['epoch'] + 1
        print('\nLoaded checkpoint from epoch %d.\n' % start_epoch)
        model = checkpoint['model']
        optimizer = checkpoint['optimizer']
    
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(start_epoch, epochs):
        
        if epoch == 100:
            optimizer = optim.SGD(model.parameters(), lr=LR*0.1, momentum=0.9, weight_decay=5e-4)
        if epoch == 150:
            optimizer = optim.SGD(model.parameters(), lr=LR*0.01, momentum=0.9, weight_decay=5e-4)
        
        train(train_loader = train_loader,
             model = model,
             criterion=criterion,
             optimizer=optimizer,
             epoch=epoch)
        save_checkpoint(epoch, model, optimizer)
        evaluate(test_loader, model)
        

def train(train_loader, model, criterion, optimizer, epoch):
    
    model = model.train()
    
    batch_time = AverageMeter()  # forward prop. + back prop. time
    data_time = AverageMeter()  # data loading time
    losses = AverageMeter()  # loss

    start = time.time()
    total = 0
    correct = 0
    # 作用cutout
    cut = Cutout(n_holes = 1, length = 16)
    for i, data in enumerate(train_loader, 0):
        length = len(train_loader)
        inputs, labels = data
        for dd in range(inputs.size(0)):
            if random.random()<0.5:
                inputs[dd] = cut(inputs[dd])
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        # forward + backward
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        losses.update(loss.item(), inputs.size(0))
        batch_time.update(time.time() - start)
        start = time.time()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += predicted.eq(labels.data).cpu().sum()
        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(epoch, i, len(train_loader),
                                                    batch_time=batch_time,
                                                    data_time=data_time, loss=losses))
            f1 = open("train_odenet18_cutout.txt", "a")
            f1.write('Epoch: [{0}][{1}/{2}]\t'
                  'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t\n'.format(epoch, i, len(train_loader),
                                                    batch_time=batch_time,
                                                    data_time=data_time, loss=losses))
            f1.close()
    print(correct/total)
    f2 = open("train_acc_odenet18_cutout.txt", "a")
    f2.write('odenet_cifar10_cutout测试分类准确率为：%.3f%%\n' % (100 * correct / total))
    f2.close()
def evaluate(test_loader, model):
    
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for i, data in enumerate(test_loader, 0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            # 取得分最高的那个类 (outputs.data的索引号)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum()
        print('测试分类准确率为：%.3f%%' % (100 * correct / total))
        f3 = open("test_acc_odenet18_cutout.txt", "a")
        f3.write('odenet_cutout测试分类准确率为：%.3f%%\n' % (100 * correct / total))
        f3.close()
        acc = 100. * correct / total
        best_acc = 85
        if acc > best_acc:
            f3 = open("best_acc_odenet18_cutout.txt", "w")
            f3.write("EPOCH=%d,best_acc= %.3f%%" % (epoch + 1, acc))
            f3.close()
            best_acc = acc
if __name__ == '__main__':
    main()


Loaded checkpoint from epoch 108.

Epoch: [108][0/391]	Batch Time 6.974 (6.974)	Data Time 0.000 (0.000)	Loss 0.0075 (0.0075)	
Epoch: [108][20/391]	Batch Time 1.265 (1.525)	Data Time 0.000 (0.000)	Loss 0.0280 (0.0394)	
Epoch: [108][40/391]	Batch Time 1.265 (1.395)	Data Time 0.000 (0.000)	Loss 0.0284 (0.0443)	
Epoch: [108][60/391]	Batch Time 1.250 (1.350)	Data Time 0.000 (0.000)	Loss 0.0360 (0.0448)	
Epoch: [108][80/391]	Batch Time 1.250 (1.328)	Data Time 0.000 (0.000)	Loss 0.0397 (0.0422)	
Epoch: [108][100/391]	Batch Time 1.250 (1.315)	Data Time 0.000 (0.000)	Loss 0.0193 (0.0421)	
Epoch: [108][120/391]	Batch Time 1.250 (1.305)	Data Time 0.000 (0.000)	Loss 0.0489 (0.0420)	
Epoch: [108][140/391]	Batch Time 1.250 (1.299)	Data Time 0.000 (0.000)	Loss 0.0306 (0.0428)	
Epoch: [108][160/391]	Batch Time 1.250 (1.294)	Data Time 0.000 (0.000)	Loss 0.0771 (0.0419)	
Epoch: [108][180/391]	Batch Time 1.265 (1.291)	Data Time 0.000 (0.000)	Loss 0.0669 (0.0415)	
Epoch: [108][200/391]	Batch Time 1.250 (

Epoch: [112][140/391]	Batch Time 1.265 (1.286)	Data Time 0.000 (0.000)	Loss 0.0350 (0.0390)	
Epoch: [112][160/391]	Batch Time 1.265 (1.284)	Data Time 0.000 (0.000)	Loss 0.0395 (0.0398)	
Epoch: [112][180/391]	Batch Time 1.281 (1.282)	Data Time 0.000 (0.000)	Loss 0.0281 (0.0400)	
Epoch: [112][200/391]	Batch Time 1.265 (1.281)	Data Time 0.000 (0.000)	Loss 0.0505 (0.0399)	
Epoch: [112][220/391]	Batch Time 1.265 (1.280)	Data Time 0.000 (0.000)	Loss 0.0838 (0.0405)	
Epoch: [112][240/391]	Batch Time 1.281 (1.279)	Data Time 0.000 (0.000)	Loss 0.0092 (0.0402)	
Epoch: [112][260/391]	Batch Time 1.265 (1.279)	Data Time 0.000 (0.000)	Loss 0.0070 (0.0400)	
Epoch: [112][280/391]	Batch Time 1.265 (1.278)	Data Time 0.000 (0.000)	Loss 0.0803 (0.0401)	
Epoch: [112][300/391]	Batch Time 1.281 (1.277)	Data Time 0.000 (0.000)	Loss 0.0227 (0.0398)	
Epoch: [112][320/391]	Batch Time 1.265 (1.277)	Data Time 0.000 (0.000)	Loss 0.0318 (0.0396)	
Epoch: [112][340/391]	Batch Time 1.265 (1.277)	Data Time 0.000 (0.000)

Epoch: [116][280/391]	Batch Time 1.265 (1.277)	Data Time 0.000 (0.000)	Loss 0.0480 (0.0385)	
Epoch: [116][300/391]	Batch Time 1.280 (1.277)	Data Time 0.000 (0.000)	Loss 0.0406 (0.0383)	
Epoch: [116][320/391]	Batch Time 1.265 (1.276)	Data Time 0.000 (0.000)	Loss 0.0431 (0.0379)	
Epoch: [116][340/391]	Batch Time 1.265 (1.275)	Data Time 0.000 (0.000)	Loss 0.0532 (0.0379)	
Epoch: [116][360/391]	Batch Time 1.281 (1.275)	Data Time 0.000 (0.000)	Loss 0.0759 (0.0379)	
Epoch: [116][380/391]	Batch Time 1.265 (1.275)	Data Time 0.000 (0.000)	Loss 0.0215 (0.0379)	
tensor(0.9874)
测试分类准确率为：93.880%
Epoch: [117][0/391]	Batch Time 3.841 (3.841)	Data Time 0.000 (0.000)	Loss 0.0174 (0.0174)	
Epoch: [117][20/391]	Batch Time 1.265 (1.388)	Data Time 0.000 (0.000)	Loss 0.0250 (0.0269)	
Epoch: [117][40/391]	Batch Time 1.281 (1.330)	Data Time 0.000 (0.000)	Loss 0.0186 (0.0293)	
Epoch: [117][60/391]	Batch Time 1.265 (1.309)	Data Time 0.000 (0.000)	Loss 0.0267 (0.0316)	
Epoch: [117][80/391]	Batch Time 1.250 (1.29

Epoch: [121][20/391]	Batch Time 1.281 (1.391)	Data Time 0.000 (0.000)	Loss 0.0227 (0.0495)	
Epoch: [121][40/391]	Batch Time 1.276 (1.332)	Data Time 0.000 (0.000)	Loss 0.0576 (0.0414)	
Epoch: [121][60/391]	Batch Time 1.265 (1.313)	Data Time 0.000 (0.000)	Loss 0.0421 (0.0420)	
Epoch: [121][80/391]	Batch Time 1.281 (1.303)	Data Time 0.000 (0.000)	Loss 0.0332 (0.0391)	
Epoch: [121][100/391]	Batch Time 1.265 (1.296)	Data Time 0.000 (0.000)	Loss 0.0428 (0.0401)	
Epoch: [121][120/391]	Batch Time 1.265 (1.292)	Data Time 0.000 (0.000)	Loss 0.0390 (0.0393)	
Epoch: [121][140/391]	Batch Time 1.281 (1.290)	Data Time 0.000 (0.000)	Loss 0.0209 (0.0381)	
Epoch: [121][160/391]	Batch Time 1.265 (1.287)	Data Time 0.000 (0.000)	Loss 0.0294 (0.0376)	
Epoch: [121][180/391]	Batch Time 1.281 (1.286)	Data Time 0.000 (0.000)	Loss 0.0593 (0.0363)	
Epoch: [121][200/391]	Batch Time 1.265 (1.284)	Data Time 0.000 (0.000)	Loss 0.0582 (0.0364)	
Epoch: [121][220/391]	Batch Time 1.265 (1.283)	Data Time 0.000 (0.000)	Los

Epoch: [125][180/391]	Batch Time 1.265 (1.285)	Data Time 0.000 (0.000)	Loss 0.0457 (0.0322)	
Epoch: [125][200/391]	Batch Time 1.281 (1.283)	Data Time 0.000 (0.000)	Loss 0.0298 (0.0315)	
Epoch: [125][220/391]	Batch Time 1.259 (1.282)	Data Time 0.000 (0.000)	Loss 0.0336 (0.0317)	
Epoch: [125][240/391]	Batch Time 1.265 (1.281)	Data Time 0.000 (0.000)	Loss 0.0405 (0.0319)	
Epoch: [125][260/391]	Batch Time 1.281 (1.281)	Data Time 0.000 (0.000)	Loss 0.0182 (0.0321)	
Epoch: [125][280/391]	Batch Time 1.265 (1.280)	Data Time 0.000 (0.000)	Loss 0.0396 (0.0321)	
Epoch: [125][300/391]	Batch Time 1.265 (1.279)	Data Time 0.000 (0.000)	Loss 0.0118 (0.0322)	
Epoch: [125][320/391]	Batch Time 1.281 (1.279)	Data Time 0.000 (0.000)	Loss 0.0356 (0.0322)	
Epoch: [125][340/391]	Batch Time 1.265 (1.279)	Data Time 0.000 (0.000)	Loss 0.0406 (0.0327)	
Epoch: [125][360/391]	Batch Time 1.265 (1.278)	Data Time 0.000 (0.000)	Loss 0.0145 (0.0325)	
Epoch: [125][380/391]	Batch Time 1.274 (1.278)	Data Time 0.000 (0.000)

...

tensor(0.9919)
测试分类准确率为：93.970%
Epoch: [182][0/391]	Batch Time 3.807 (3.807)	Data Time 0.000 (0.000)	Loss 0.0500 (0.0500)	
Epoch: [182][20/391]	Batch Time 1.250 (1.385)	Data Time 0.000 (0.000)	Loss 0.0315 (0.0268)	
Epoch: [182][40/391]	Batch Time 1.265 (1.326)	Data Time 0.000 (0.000)	Loss 0.0099 (0.0254)	
Epoch: [182][60/391]	Batch Time 1.265 (1.306)	Data Time 0.000 (0.000)	Loss 0.0179 (0.0258)	
Epoch: [182][80/391]	Batch Time 1.265 (1.296)	Data Time 0.000 (0.000)	Loss 0.0308 (0.0246)	
Epoch: [182][100/391]	Batch Time 1.250 (1.290)	Data Time 0.000 (0.000)	Loss 0.0225 (0.0253)	
Epoch: [182][120/391]	Batch Time 1.265 (1.286)	Data Time 0.000 (0.000)	Loss 0.0339 (0.0251)	
Epoch: [182][140/391]	Batch Time 1.265 (1.283)	Data Time 0.000 (0.000)	Loss 0.0458 (0.0243)	
Epoch: [182][160/391]	Batch Time 1.265 (1.281)	Data Time 0.000 (0.000)	Loss 0.0026 (0.0247)	
Epoch: [182][180/391]	Batch Time 1.265 (1.279)	Data Time 0.000 (0.000)	Loss 0.0032 (0.0252)	
Epoch: [182][200/391]	Batch Time 1.254 (1.27

Epoch: [186][140/391]	Batch Time 1.265 (1.283)	Data Time 0.000 (0.000)	Loss 0.0318 (0.0248)	
Epoch: [186][160/391]	Batch Time 1.265 (1.281)	Data Time 0.000 (0.000)	Loss 0.0129 (0.0253)	
Epoch: [186][180/391]	Batch Time 1.265 (1.279)	Data Time 0.000 (0.000)	Loss 0.0193 (0.0246)	
Epoch: [186][200/391]	Batch Time 1.265 (1.278)	Data Time 0.000 (0.000)	Loss 0.0297 (0.0240)	
Epoch: [186][220/391]	Batch Time 1.265 (1.277)	Data Time 0.000 (0.000)	Loss 0.0533 (0.0234)	
Epoch: [186][240/391]	Batch Time 1.265 (1.276)	Data Time 0.000 (0.000)	Loss 0.0224 (0.0234)	
Epoch: [186][260/391]	Batch Time 1.265 (1.275)	Data Time 0.000 (0.000)	Loss 0.0143 (0.0237)	
Epoch: [186][280/391]	Batch Time 1.265 (1.274)	Data Time 0.000 (0.000)	Loss 0.0057 (0.0237)	
Epoch: [186][300/391]	Batch Time 1.265 (1.274)	Data Time 0.000 (0.000)	Loss 0.0492 (0.0237)	
Epoch: [186][320/391]	Batch Time 1.265 (1.273)	Data Time 0.000 (0.000)	Loss 0.0135 (0.0233)	
Epoch: [186][340/391]	Batch Time 1.261 (1.273)	Data Time 0.000 (0.000)

Epoch: [190][280/391]	Batch Time 1.265 (1.274)	Data Time 0.000 (0.000)	Loss 0.0055 (0.0269)	
Epoch: [190][300/391]	Batch Time 1.265 (1.274)	Data Time 0.000 (0.000)	Loss 0.0222 (0.0269)	
Epoch: [190][320/391]	Batch Time 1.265 (1.273)	Data Time 0.000 (0.000)	Loss 0.0103 (0.0273)	
Epoch: [190][340/391]	Batch Time 1.265 (1.273)	Data Time 0.000 (0.000)	Loss 0.0284 (0.0269)	
Epoch: [190][360/391]	Batch Time 1.265 (1.272)	Data Time 0.000 (0.000)	Loss 0.0145 (0.0265)	
Epoch: [190][380/391]	Batch Time 1.265 (1.272)	Data Time 0.000 (0.000)	Loss 0.0897 (0.0271)	
tensor(0.9919)
测试分类准确率为：94.010%
Epoch: [191][0/391]	Batch Time 3.825 (3.825)	Data Time 0.000 (0.000)	Loss 0.0086 (0.0086)	
Epoch: [191][20/391]	Batch Time 1.250 (1.387)	Data Time 0.000 (0.000)	Loss 0.0144 (0.0203)	
Epoch: [191][40/391]	Batch Time 1.269 (1.328)	Data Time 0.000 (0.000)	Loss 0.0191 (0.0239)	
Epoch: [191][60/391]	Batch Time 1.265 (1.307)	Data Time 0.000 (0.000)	Loss 0.0033 (0.0246)	
Epoch: [191][80/391]	Batch Time 1.250 (1.29

Epoch: [195][20/391]	Batch Time 1.265 (1.386)	Data Time 0.000 (0.000)	Loss 0.0277 (0.0311)	
Epoch: [195][40/391]	Batch Time 1.265 (1.327)	Data Time 0.000 (0.000)	Loss 0.0140 (0.0294)	
Epoch: [195][60/391]	Batch Time 1.278 (1.307)	Data Time 0.000 (0.000)	Loss 0.0138 (0.0258)	
Epoch: [195][80/391]	Batch Time 1.265 (1.296)	Data Time 0.000 (0.000)	Loss 0.0126 (0.0246)	
Epoch: [195][100/391]	Batch Time 1.265 (1.290)	Data Time 0.000 (0.000)	Loss 0.0146 (0.0247)	
Epoch: [195][120/391]	Batch Time 1.265 (1.286)	Data Time 0.000 (0.000)	Loss 0.0160 (0.0245)	
Epoch: [195][140/391]	Batch Time 1.265 (1.283)	Data Time 0.000 (0.000)	Loss 0.0509 (0.0247)	
Epoch: [195][160/391]	Batch Time 1.265 (1.281)	Data Time 0.000 (0.000)	Loss 0.0211 (0.0239)	
Epoch: [195][180/391]	Batch Time 1.265 (1.279)	Data Time 0.000 (0.000)	Loss 0.0088 (0.0242)	
Epoch: [195][200/391]	Batch Time 1.265 (1.278)	Data Time 0.000 (0.000)	Loss 0.0223 (0.0241)	
Epoch: [195][220/391]	Batch Time 1.265 (1.277)	Data Time 0.000 (0.000)	Los

Epoch: [199][180/391]	Batch Time 1.265 (1.280)	Data Time 0.000 (0.000)	Loss 0.0301 (0.0231)	
Epoch: [199][200/391]	Batch Time 1.265 (1.278)	Data Time 0.000 (0.000)	Loss 0.0396 (0.0239)	
Epoch: [199][220/391]	Batch Time 1.265 (1.277)	Data Time 0.000 (0.000)	Loss 0.0233 (0.0245)	
Epoch: [199][240/391]	Batch Time 1.265 (1.276)	Data Time 0.000 (0.000)	Loss 0.0191 (0.0246)	
Epoch: [199][260/391]	Batch Time 1.265 (1.275)	Data Time 0.000 (0.000)	Loss 0.0437 (0.0242)	
Epoch: [199][280/391]	Batch Time 1.265 (1.275)	Data Time 0.000 (0.000)	Loss 0.0191 (0.0239)	
Epoch: [199][300/391]	Batch Time 1.265 (1.274)	Data Time 0.000 (0.000)	Loss 0.0178 (0.0241)	
Epoch: [199][320/391]	Batch Time 1.265 (1.274)	Data Time 0.000 (0.000)	Loss 0.0090 (0.0239)	
Epoch: [199][340/391]	Batch Time 1.269 (1.273)	Data Time 0.000 (0.000)	Loss 0.0341 (0.0244)	
Epoch: [199][360/391]	Batch Time 1.265 (1.273)	Data Time 0.000 (0.000)	Loss 0.0131 (0.0243)	
Epoch: [199][380/391]	Batch Time 1.265 (1.272)	Data Time 0.000 (0.000)